# 1. Load Model

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import snapshot_download
from tuned_lens import TunedLens
from tuned_lens.plotting import PredictionTrajectory
import numpy as np
import os

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it", device_map="auto")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2-2b-it", 
                                                   device_map="auto", 
                                                   torch_dtype=torch.bfloat16)

# Note that we use right-padding, since here we are only focusing on the encoding not generation.
tokenizer.padding_side = "right"
if tokenizer.pad_token is None:
    # Safe choice: use eos as pad if no dedicated pad token
    tokenizer.pad_token = tokenizer.eos_token

model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (post_attention_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (pre_feedforward_layernorm): Gemm

# 2. Prepare Prompts

In [3]:
# from generate_prompts import return_model_prompts
# prompts_all_parallel_multiple = return_model_prompts(model_name = "google/gemma-2-9b-it",
#                                                      data_name = 'BFCL_v4_live_parallel_multiple.json')

In [4]:
# import json
# with open('../SAE_Summ/prompts_with_generation_cnndm.json', 'r') as file:
#     prompts_with_generation_cnndm = json.load(file)

In [5]:
# import json
# with open('../SAE_Summ/prompts_with_generation_all_extractive_cnndm.json', 'r') as file:
#     prompts_with_generation_all_extractive_cnndm = json.load(file)

In [6]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    np.random.seed(seed)  # Numpy module.
    random.seed(seed)  # Python random module.
    # torch.set_deterministic(True)
    torch.backends.cudnn.enabled = False
    torch.backends.cudnn.benchmark = False
    os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'
    os.environ['PYTHONHASHSEED'] = str(seed)

In [7]:
all_prompts = {}

# 2.1 AGNews

In [8]:
from datasets import load_dataset
import random 

dataset = load_dataset('ag_news', split=['train', 'test'])
train_data_agnews = dataset[0]
test_data_agnews = dataset[1]

In [9]:
num_examples_per_label = 500

label_demonstrations = {i:[] for i in range(4)}

for i in range(len(train_data_agnews)):
    label_demonstrations[train_data_agnews[i]['label']].append(i)
    
set_seed(42)
test_query_indices = []
for i in range(len(label_demonstrations)):
    test_query_indices.extend(random.sample(label_demonstrations[i], num_examples_per_label))

In [10]:
all_prompts['agnews'] = {}

# N is the total number of few-shot examples across all labels
# for N in range(25):  # Covers 0 to 24
for N in [16]:
    all_prompts['agnews'][N] = []
    num_labels = len(label_demonstrations)
    
    for i, tq_index in enumerate(test_query_indices):
        set_seed(i)
        
        # 1. Use the query index 'i' to shift the starting label
        # This ensures that for N=1, Query 0 gets Label 0, Query 1 gets Label 1, etc.
        start_label_offset = i % num_labels
        
        shots_per_label = [N // num_labels] * num_labels
        remainder = N % num_labels
        
        # Distribute the remainder starting from the offset
        for r in range(remainder):
            label_to_increment = (start_label_offset + r) % num_labels
            shots_per_label[label_to_increment] += 1
            
        fse_indices = []
        
        # 2. Sample from each label
        for l in range(num_labels):
            needed = shots_per_label[l]
            if needed == 0:
                continue
                
            available_pool = [idx for idx in label_demonstrations[l] if idx != tq_index]
            sampled = random.sample(available_pool, needed)
            fse_indices.extend(sampled)
            
        # 3. Randomize the order of shots within the prompt
        random.shuffle(fse_indices)
        
        prompt = 'Pretend that you are an expert in news topic classification. For a given news article, you have to assess the topic, determining whether it is world, sports, business, or technology.'

        for j, fse_index in enumerate(fse_indices):
            prompt += f'\n\nExample {j+1}'
            prompt += f"\nArticle:\n{train_data_agnews[fse_index]['text']}"
            prompt += f"\nTopic:\n{train_data_agnews.features['label'].names[train_data_agnews[fse_index]['label']]}"

        prompt += f'\n\nExample {len(fse_indices)+1}'
        prompt += f"\nArticle:\n{train_data_agnews[tq_index]['text']}"
        prompt += f"\nTopic:\n{train_data_agnews.features['label'].names[train_data_agnews[tq_index]['label']]}"


        all_prompts['agnews'][N].append("<bos><start_of_turn>user\n"+prompt.replace('Sci/Tech', 'Technology'))


In [11]:
# for example_idx in range(len(all_prompts['agnews'][0])):
#     p0 = all_prompts['agnews'][0][example_idx].split('Article:')[1]
#     for N in all_prompts['agnews'].keys():
#         assert p0 == all_prompts['agnews'][N][example_idx].split('Article:')[-1]

# Load SAEs

In [12]:
def clear_all_hooks(model: torch.nn.Module):
    for module in model.modules():
        module._forward_hooks.clear()
        module._forward_pre_hooks.clear()
        module._backward_hooks.clear()

In [13]:
import torch

@torch.no_grad()
def sae_logits_before_jumprelu(sae, x: torch.Tensor) -> torch.Tensor:
    """
    Returns encoder pre-activations ("logits") for a SAELens JumpReLUSAE.

    x can be shape:
      - (d_in,)
      - (seq, d_in)
      - (batch, seq, d_in)

    Output will be the same shape but with last dim = d_sae.
    """
    # Ensure correct device/dtype (optional but usually helpful)
    x = x.to(device=sae.device, dtype=sae.dtype)

    # Use SAELens preprocessing (handles b_dec-to-input and any norm/reshape logic)
    sae_in = sae.process_sae_in(x)

    # Pre-activations (this is what you want)
    hidden_pre = sae_in @ sae.W_enc + sae.b_enc

    # Match SAELens exactly if hooks are present (usually identity unless you attached hooks)
    if hasattr(sae, "hook_sae_acts_pre"):
        hidden_pre = sae.hook_sae_acts_pre(hidden_pre)

    return hidden_pre


In [14]:
@torch.no_grad()
def extract_decoding_embeddings(sae, layer_idx):
    def hook(module, inputs, output):
        # For Residual SAEs: output[0] is [Batch, Seq, Hidden]
        x = output[0]
        
        global feature_summary_trajectories
        
        with torch.no_grad():
            # logits shape: [Batch, Seq, Num_Features]
            logits = sae_logits_before_jumprelu(sae, x)
            
            for b, ex_idx in enumerate(CURRENT_BATCH_EXAMPLE_IDXS):
                start_pos = example_summary[task_type][N][ex_idx]['analysis_start']
                end_pos = lengths[b]
                
                xxx = logits[b, start_pos:end_pos].clone().cpu().float() # [Seq, Feat]
                feature_summary_trajectories[task_type][N].append(xxx)
                
        return output
    return hook

In [15]:
from sae_lens import SAE

clear_all_hooks(model)

sae_set = {}
target_layer = 18

for layer_idx in range(target_layer,target_layer+1):
    sae, sae_config, sparsity = SAE.from_pretrained(
        release="gemma-scope-2b-pt-res-canonical", 
        sae_id=f"layer_{layer_idx}/width_16k/canonical", 
        device="cuda")
    
    sae_set[layer_idx] = sae
    target_steering_block = model.get_submodule(f"model.layers.{layer_idx}")
    target_steering_block._forward_hooks.clear()
    steering_hook = target_steering_block.register_forward_hook(extract_decoding_embeddings(sae=sae, layer_idx = layer_idx))

/var/tmp/ipykernel_68542/4280670331.py:9: DeprecationWarning: Unpacking SAE objects is deprecated. SAE.from_pretrained() now returns only the SAE object. Use SAE.from_pretrained_with_cfg_and_sparsity() to get the config dict and sparsity as well.
  sae, sae_config, sparsity = SAE.from_pretrained(


# 1. Using AGNews

In [16]:
feature_summary_peak_max = {}
feature_summary_peak_min = {}
feature_summary_trajectories = {}
example_summary = {}

In [17]:
task_type = 'agnews'

# General

In [18]:
BATCH_SIZE = 4

feature_summary_peak_max[task_type] = {}
feature_summary_peak_min[task_type] = {}
feature_summary_trajectories[task_type] = {}
example_summary[task_type] = {}


# for N in range(0, 20, 2):
# for N in range(0, 24, 3):
for N in [16]:
    example_summary[task_type][N] = {}
    
    num_examples = len(all_prompts[task_type][N])
    
    feature_summary_peak_max[task_type][N] = []
    feature_summary_peak_min[task_type][N] = []
    feature_summary_trajectories[task_type][N] = []
    
    for batch_start in range(0, num_examples, BATCH_SIZE):
        batch_prompts = all_prompts[task_type][N][batch_start: batch_start + BATCH_SIZE]
        batch_size_actual = len(batch_prompts)

        print(f"{task_type}, {N}-shot, batch_start={batch_start}, batch_size={batch_size_actual}")

        if batch_start == 0:
            print("First prompt in this N-shot setting:\n", batch_prompts[0])

        enc = tokenizer(
            batch_prompts,
            return_tensors="pt",
            padding=True,               # <-- batching
            truncation=False,
            add_special_tokens=False
        )
        input_ids = enc["input_ids"].to(model.device)
        attention_mask = enc["attention_mask"].to(model.device)

        res = {
            "input_ids": input_ids,
            "input_masks": attention_mask,
        }
        
        lengths = attention_mask.sum(dim=1).tolist()
        
        for b in range(batch_size_actual):
            example_idx = batch_start + b
            L = lengths[b]

            decoded_tokens = [
                tokenizer.decode(int(input_ids[b, j]))
                for j in range(L)
            ]
        
            Example_start = []
            analysis_start = 0
            for j in range(max(0, L - 2)):
                if (
                    decoded_tokens[j] == 'Article'
                    and decoded_tokens[j + 1] == ':'
                    and decoded_tokens[j + 2] == '\n'
                ):
                    Example_start.append(j)
                    
                if (
                    decoded_tokens[j] == 'Example'
                    and decoded_tokens[j+1] == '1'
                ):
                    analysis_start = j
                    
            example_summary[task_type][N][example_idx] = {'All_tokens': L,
                                                          'analysis_start': Example_start[-1],
                                                          'tq_label': decoded_tokens[-1]}
            if example_idx == 0:
                print("Example summary for example_idx=0:\n", example_summary[task_type][N][example_idx])

                
                
        CURRENT_BATCH_EXAMPLE_IDXS = list(range(batch_start, batch_start + batch_size_actual))
    
        with torch.no_grad():
            outputs = model.generate(
                res["input_ids"],
                attention_mask=res["input_masks"],
                max_new_tokens=1,
                do_sample=False,
                pad_token_id=tokenizer.pad_token_id,
            )

agnews, 16-shot, batch_start=0, batch_size=4
First prompt in this N-shot setting:
 <bos><start_of_turn>user
Pretend that you are an expert in news topic classification. For a given news article, you have to assess the topic, determining whether it is world, sports, business, or technology.

Example 1
Article:
Pakistan and India discuss missiles, hotlines Pakistan and India held a second day of talks on Wednesday to agree on steps to avert misunderstandings over their nuclear arsenals, a Pakistani official said.
Topic:
World

Example 2
Article:
Vic koalas to go on the pill Victorian environment officers are about to embark on an ambitious campaign, implanting contraceptives in 2,000 koalas. The new koala management strategy is designed to protect the koalas and their environment.
Topic:
Technology

Example 3
Article:
Automakers Tout Hybrids, Power Rules General Motors Corp., which sells  the gas-gobbling Hummer, urges Americans to "Get Green" on a special Web site and is producing adver

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


agnews, 16-shot, batch_start=4, batch_size=4


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


agnews, 16-shot, batch_start=8, batch_size=4


...

agnews, 16-shot, batch_start=1988, batch_size=4


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


agnews, 16-shot, batch_start=1992, batch_size=4


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


agnews, 16-shot, batch_start=1996, batch_size=4


In [19]:
import torch

def get_all_feature_negations(W_dec):
    """
    Returns a dictionary mapping EVERY feature to its most negative partner.
    """
    # 1. Normalize
    norms = torch.norm(W_dec, dim=1, keepdim=True)
    W_norm = W_dec / (norms + 1e-8)
    
    # 2. Compute Similarity Matrix
    # [16384, 16384]
    cos_sim_matrix = torch.mm(W_norm, W_norm.t())
    
    # 3. Mask the diagonal (Self-similarity is 1.0, we want negatives)
    # We fill diagonal with infinity so it doesn't interfere with finding the min
    cos_sim_matrix.fill_diagonal_(float('inf'))
    
    # 4. Find the min (most negative) for EVERY row at once
    # values: [16384] (the correlation score)
    # indices: [16384] (the index of the partner feature)
    min_values, min_indices = torch.min(cos_sim_matrix, dim=1)
    
    # 5. Build the dictionary
    all_features_map = {}
    
    for i in range(W_dec.shape[0]):
        all_features_map[i] = {
            'pair_feature': min_indices[i].item(),
            'sim': min_values[i].item()
        }
        
    print(f"Processed {len(all_features_map)} features.")
    return all_features_map

In [20]:
all_feature_map = get_all_feature_negations(sae.W_dec)

Processed 16384 features.


---

# Slightly updated our CP-search metric to "Resample and Measure Consistency" approach. Slightly easier to identify cyclic features.

In [73]:
import torch
import torch.nn.functional as F

try:
    from tqdm.auto import tqdm
except ImportError:
    def tqdm(x, **kwargs):  # fallback
        return x

@torch.no_grad()
def compute_profile_consistency_scores(
    feature_summary_trajectories,
    task_type,
    N,
    grid_size=50,
    example_limit=None,      # e.g., 800 or None for all
    device=None,
    min_norm=1e-6,           # skip near-constant profiles
    center="mean",           # "mean" or "none"
    align_corners=True,
):
    """
    Returns:
      pcs:   [F] float tensor in [-1,1] (higher = more consistent profile)
      count: [F] int tensor (#examples used per feature)
      mean_profile_unit: [F, G] mean of unit profiles (useful for visualization)
    """
    examples = feature_summary_trajectories[task_type][N]
    if isinstance(examples, dict):
        examples = list(examples.values())
    if example_limit is not None:
        examples = examples[:example_limit]

    # Infer feature dim
    T0, Ftrs = examples[0].shape
    if device is None:
        device = examples[0].device

    # Accumulators
    sum_u = torch.zeros((Ftrs, grid_size), device=device, dtype=torch.float32)
    count = torch.zeros((Ftrs,), device=device, dtype=torch.int32)

    for feats in tqdm(examples, desc=f"Resample+accumulate (N={N})"):
        # feats: [T, F]
        feats = feats.to(device)
        if feats.size(0) < 2:
            continue

        # [T,F] -> [1,F,T] for interpolate
        x = feats.transpose(0, 1).unsqueeze(0).float()  # [1, F, T]

        # Resample time axis to grid_size: [1,F,G] -> [F,G]
        xg = F.interpolate(x, size=grid_size, mode="linear", align_corners=align_corners)
        xg = xg.squeeze(0)  # [F, G]

        # Center each feature's profile
        if center == "mean":
            xg = xg - xg.mean(dim=1, keepdim=True)
        elif center == "none":
            pass
        else:
            raise ValueError("center must be 'mean' or 'none'")

        # Normalize per feature to unit norm (so this is shape-only consistency)
        norms = torch.linalg.norm(xg, dim=1)  # [F]
        mask = norms > min_norm               # [F]

        # In-place normalization + zero-out degenerate
        xg[mask] = xg[mask] / norms[mask].unsqueeze(1)
        xg[~mask] = 0.0

        sum_u += xg
        count += mask.to(count.dtype)

    c = count.float()
    valid = c > 1

    # ||sum u||^2
    sum_norm2 = (sum_u * sum_u).sum(dim=1)  # [F]

    # Mean pairwise cosine similarity (efficient)
    pcs = torch.zeros((Ftrs,), device=device, dtype=torch.float32)
    pcs[valid] = (sum_norm2[valid] - c[valid]) / (c[valid] * (c[valid] - 1.0) + 1e-12)

    mean_profile_unit = sum_u / c.clamp_min(1.0).unsqueeze(1)  # [F,G]

    return pcs.detach().cpu(), count.detach().cpu(), mean_profile_unit.detach().cpu()

In [74]:
import torch
import torch.nn.functional as F

def get_consistent_features_final(trajectories_list, grid_size=50, batch_size=512):
    num_features = trajectories_list[0].shape[1]
    num_examples = len(trajectories_list)
    
    # 1. Resample trajectories (Linear Interpolation)
    resampled_list = []
    for traj in trajectories_list:
        # Shape for interpolate: [1, Features, Length]
        t = traj.t().unsqueeze(0) 
        resampled = F.interpolate(t, size=grid_size, mode='linear', align_corners=True)
        resampled_list.append(resampled.squeeze(0)) 

    # [num_features, num_examples, grid_size]
    feature_data = torch.stack(resampled_list).permute(1, 0, 2)
    
    consistency_scores = torch.zeros(num_features, device=feature_data.device)

    # 2. Vectorized logic WITHOUT using torch.bmm
    for i in range(0, num_features, batch_size):
        end_idx = min(i + batch_size, num_features)
        batch = feature_data[i:end_idx] # [Batch, Examples, Grid]

        # Step A: Normalize (Pearson Correlation requires Mean=0, Norm=1)
        means = batch.mean(dim=2, keepdim=True)
        centered = batch - means
        norms = torch.norm(centered, dim=2, keepdim=True)
        
        # Avoid division by zero
        norms = torch.where(norms == 0, torch.tensor(1.0, device=norms.device), norms)
        normalized = centered / norms 

        # Step B: The Mathematical Shortcut
        # Instead of calculating every pair (i, j) and then summing, 
        # we sum the vectors FIRST and then square the result.
        
        # 1. Sum all examples together: [Batch, Grid]
        sum_of_examples = normalized.sum(dim=1) 
        
        # 2. Square the sum and total it up: [Batch]
        # This is equivalent to summing ALL pairs (including i*i)
        total_sum_sq = (sum_of_examples**2).sum(dim=1)
        
        # 3. Subtract the "self-correlations" (where i == j)
        # Since we normalized to unit length, each self-dot-product is exactly 1.0.
        # There are 'num_examples' such pairs.
        only_off_diagonal_sum = total_sum_sq - num_examples
        
        # 4. Average by the number of pairs: N * (N - 1)
        batch_scores = only_off_diagonal_sum / (num_examples * (num_examples - 1))
        
        consistency_scores[i:end_idx] = batch_scores

    return torch.nan_to_num(consistency_scores, nan=0.0)

In [22]:
# trajectories = feature_summary_trajectories[task_type][N]
# scores = get_consistent_features_final(trajectories)

# # Get indices of the top 10 most consistent features
# top_consistent_indices = torch.argsort(scores, descending=True)[:10]

In [24]:
# torch.save(scores, f'G2B_L{target_layer}_{task_type}.pt')
# torch.save(scores, f'G9B_L{target_layer}_{task_type}.pt')

In [ ]:
# pcs, count, mean_prof = compute_profile_consistency_scores(
#     feature_summary_trajectories,
#     task_type=task_type,
#     N=16,
#     grid_size=50,
#     example_limit=2000, 
#     center = "mean"
# )
# topk = torch.topk(pcs, k=20)
# print("Top-20 features by profile consistency:", topk.indices.tolist())
# print("Top-20 PCS scores:", topk.values.tolist())

# For Plotting

In [27]:
@torch.no_grad()
def extract_sae_results_for_plotting(sae, layer_idx):
    def hook(module, inputs, output):
        x = output[0]
        
        global plot_result
        global plot_result_logits
        
        sae_encoded = sae.encode(x).float()
        logits = sae_logits_before_jumprelu(sae, x).clone().float()
        
        plot_result.append(sae_encoded[0].clone())
        plot_result_logits.append(logits[0].clone())
        return output  # do not alter the model behavior
    return hook

In [28]:
import plotly.graph_objects as go

def plot_activations_agnews(input_prompt, target_layer, example_idx, feature_idx, model_gemma, tokenizer, without_bos = False, detailed = True, bar_width=3):
    clear_all_hooks(model_gemma)
    print('='*77)
    print('feature_idx:', feature_idx)
    print('='*77)
    global plot_result
    global plot_result_logits
    
    plot_result = []
    plot_result_logits = []
    Label_Words = []
    
    target_steering_block = model_gemma.get_submodule(f"model.layers.{target_layer}")
    target_steering_block._forward_hooks.clear()
    steering_hook = target_steering_block.register_forward_hook(extract_sae_results_for_plotting(sae=sae, layer_idx = target_layer))
    
    batch_prompts = [input_prompt]
    batch_input = tokenizer.batch_encode_plus(batch_prompts,
                                              return_tensors='pt',
                                              add_special_tokens=False)
    batch_input = batch_input.data

    res = {
        'input_ids': batch_input['input_ids'],
        'input_masks': batch_input['attention_mask']
    }
    res = {k: v.to(model_gemma.device) for k, v in res.items()}
    
    decoded_tokens = [tokenizer.decode(res['input_ids'][0][j]) for j in range(len(res['input_ids'][0]))]
    
    output = model_gemma.generate(
            res['input_ids'],
            attention_mask=res['input_masks'],
            max_new_tokens=1,
            do_sample=False,
            return_dict_in_generate=True,  # <-- important
            output_scores=True,            # <-- gives you per-step logits
        )
    
    generation_start = len(decoded_tokens)
    
    decoded_tokens = [tokenizer.decode(item) for item in output['sequences'][0][:-1]]
    plot_result_ = torch.cat(plot_result, dim=0)
    plot_result_logits_ = torch.cat(plot_result_logits, dim=0)
    
    assert len(output['sequences'][0]) == len(plot_result_)+1
    
    plot_tensor = plot_result_[:, feature_idx]
    plot_tensor_logits = plot_result_logits_[:, feature_idx]
    
    vals, inds = torch.topk(plot_tensor, k = int(torch.sum((plot_tensor>0).float())))
    inds = inds.tolist()
    print_tokens_with_highlights(decoded_tokens, inds)
     
    if hasattr(plot_tensor, "detach"):
        plot_tensor = plot_tensor.detach().cpu().numpy()
    if hasattr(plot_tensor_logits, "detach"):
        plot_tensor_logits = plot_tensor_logits.detach().cpu().numpy()
        
    # Create the bar plot
    if without_bos == False:
        hover_texts = [
            f"Token: {decoded_tokens[i]}<br>Next Token: {decoded_tokens[i+1]}"
            for i in range(len(decoded_tokens) - 1)
        ]
        
        fig = go.Figure(
            data=[
                go.Bar(
                    x=list(range(len(decoded_tokens))),  # token positions
                    y=plot_tensor,
                    text=hover_texts,
                    hovertemplate="Token: %{text}<br>Activation: %{y:.4f}<extra></extra>",
                    marker_color="indianred",
                    width=bar_width 
                )
            ]
        )
    else:
        hover_texts = [
            f"Token: {decoded_tokens[i+1]}<br>Next Token: {decoded_tokens[i+2]}"
            for i in range(len(decoded_tokens[1:]) - 1)
        ]
        fig = go.Figure(
            data=[
                go.Bar(
                    x=list(range(len(decoded_tokens)-1)),  # token positions
                    y=plot_tensor[1:],
                    text=hover_texts,
                    hovertemplate="Token: %{text}<br>Activation: %{y:.4f}<extra></extra>",
                    marker_color="indianred",
                    width=bar_width 
                )
            ]
        )
    
    mask = torch.tensor(plot_tensor[:generation_start]) > 0
    tokens = np.array(decoded_tokens[:generation_start])
    activated_tokens = np.array(decoded_tokens[:generation_start])[mask.numpy()].tolist()
    
    activated_indices = mask.nonzero(as_tuple=True)[0].numpy()

    # Collect right next tokens => Just to see if a feature has a causal relationship with specific words.
    # E.g., if a feature that triggers label words exists => We can call this the "Say Label Word" feature !!
    
    neighbor_indices = set()
    for i in activated_indices:
        if i + 1 < len(tokens):
            neighbor_indices.add(i + 1)

    # Remove duplicates and get the tokens
    neighbor_indices = sorted(neighbor_indices)
    neighbor_tokens = tokens[neighbor_indices].tolist()

    
    activated_tokens_next = np.array(decoded_tokens[:generation_start])[mask.numpy()].tolist()
    decoded_tokens_including_last_token = [tokenizer.decode(item) for item in output['sequences'][0]]
    
    if detailed:
        print('='*99)
        print('generated tokens:', decoded_tokens_including_last_token[generation_start:])
        print('='*99)
        print()
        print('activated_tokens ENCODER:')
        print(activated_tokens)
        print()
        print('Right next to the activated tokens:')
        print(neighbor_tokens)
        print('='*99)
        print()
        print('activated_tokens DECODER:')
        mask = torch.tensor(plot_tensor[generation_start:]) > 0
        activated_tokens = np.array(decoded_tokens[generation_start:])[mask.numpy()].tolist()
        print(activated_tokens)
        print('='*99)
    
        sorted_indices = sorted(range(len(plot_tensor[:generation_start])), key=lambda i: plot_tensor[i], reverse = True)

        print()
        print('-'*77)
        for tok_id in sorted_indices:
            if plot_tensor[tok_id] > 0:
                print(f'({repr(tokenizer.decode(output["sequences"][0][tok_id]))}, {repr(tokenizer.decode(output["sequences"][0][tok_id+1]))})=> {plot_tensor[tok_id]: .2f}')
         
        print('-'*77)
        print()
        
    # Set layout details
    step = 14
    tick_positions = list(range(0, len(decoded_tokens), step))
    tick_labels = [decoded_tokens[i] for i in tick_positions]

    fig.update_layout(
        title=f"Feature {feature_idx} Activations Across Tokens (Example {example_idx})",
        xaxis_title="Token Position",
        yaxis_title="Activation Value",
        xaxis=dict(
            tickmode="array",
            tickvals=tick_positions,
            ticktext=tick_labels,
            tickangle=45
        ),
        template="plotly_white",
        height=500,
        width=900
    )
    
    Function_start = [j for j, tok in enumerate(decoded_tokens) if tok == 'Example'
                      and decoded_tokens[j-1] == '\n\n']
    
    Label_Words = [decoded_tokens[item-2] for item in Function_start[1:]]
    Label_Words += decoded_tokens[-1]
    
    print('='*77)
    for i in range(len(Label_Words)):
        if len(set(Label_Words[:i+1])) == 4:
            print(f"Label Space Completed at {i+1}") 
            break
           
    print('='*77)
    print('Label_Words in Order:')
    for i, label in enumerate(Label_Words):
        print(f"Exemplar {i+1} = {label}")
    print('='*77)
    
    fig.add_vline(
        x=0,
        line_dash="dash",
        line_color="blue",
        line_width=2,
    )
    
    fig.add_annotation(
            x=5,
            y=1,  # normalized (top)
            xref="x",
            yref="paper",  # positions relative to the figure
            text=f"T.I.",
            showarrow=False,
            font=dict(family="Times New Roman", size=11, color="blue"),  # 👈 font size here
            yshift=10
        )
    
    for i, func_start in enumerate(Function_start):
        fig.add_vline(
            x=func_start,
            line_dash="dash",
            line_color="blue",
            line_width=2,
        )
        
        fig.add_annotation(
            x=func_start+5,
            y=1,  # normalized (top)
            xref="x",
            yref="paper",  # positions relative to the figure
            text=f"E{i+1} ({Label_Words[i][:3]})",
            showarrow=False,
            font=dict(family="Times New Roman", size=8, color="blue"),  # 👈 font size here
            yshift=10
        )
    
    fig.update_layout(
        title=f"Gemma2-9B-IT, Layer {target_layer}, Feature #{feature_idx} Activations Across Tokens",
        title_x=0.5,  # centers the title
        xaxis_title="Token Position",
        yaxis_title="Activation Value",
        font=dict(family="Times New Roman", size=18),
        template="plotly_white"
    )
    fig.show()
    
    
    logits = output.scores[0]                  # (batch, vocab) since max_new_tokens=1
    probs  = F.softmax(logits, dim=-1)         # (batch, vocab)

    top_p, top_id = torch.topk(probs, k=10, dim=-1)

    print(f"top-10 next tokens:")
    for p, tid in zip(top_p[0], top_id[0]):
        tid = int(tid.item())
        tok = tokenizer.convert_ids_to_tokens(tid)   # raw token piece
        txt = tokenizer.decode([tid])                # decoded text for that single token
        print(f"{tok!r:>18}  ({txt!r:>10})  p={p.item():.6f} (logit: {logits[0][tid]:.3f})  id={tid}")
        
        
    print('='*77)
    print('Printing Logits:')
    print('='*77)
    hover_texts = [
        f"Token: {decoded_tokens[i]}<br>Next Token: {decoded_tokens[i+1]}"
        for i in range(1, len(decoded_tokens) - 1)
    ]

    fig = go.Figure(
        data=[
            go.Bar(
                x=list(range(len(decoded_tokens)))[1:],  # token positions
                y=plot_tensor_logits[1:],
                text=hover_texts,
                hovertemplate="Token: %{text}<br>Activation: %{y:.4f}<extra></extra>",
                marker_color="indianred",
                width=bar_width,
            )
        ]
    )
    
    for i, func_start in enumerate(Function_start):
        fig.add_vline(
            x=func_start,
            line_dash="dash",
            line_color="blue",
            line_width=2,
        )
        fig.add_annotation(
            x=func_start,
            y=1,  # normalized (top)
            xref="x",
            yref="paper",  # positions relative to the figure
            text=f"E{i+1} ({Label_Words[i][:3]})",
            showarrow=False,
            font=dict(family="Times New Roman", size=8, color="blue"),  # 👈 font size here
            yshift=10
        )
    
    fig.update_layout(
        title=f"Gemma2-9B-IT, Layer {target_layer}, Feature #{feature_idx} Logits across Tokens",
        title_x=0.5,  # centers the title
        xaxis_title="Token Position",
        yaxis_title="Logits Value",
        font=dict(family="Times New Roman", size=18),
        template="plotly_white"
    )
    fig.show()
    